In [3]:
import pandas as pd
import os
import time
import numpy as np

In [4]:
fpath = os.getcwd()  # assuming df is in current dir
df = pd.read_csv("12Z_soundings.csv")
print(df.head())

   Unnamed: 0  pressure  height  temperature  dewpoint  direction  speed  \
0           0     966.0     379         10.0       3.0      145.0    2.0   
1           1     959.0     440         11.0       3.0      166.0    5.0   
2           2     939.8     610         10.2       1.6      225.0   12.0   
3           3     925.0     743          9.6       0.6      255.0    8.0   
4           4     905.9     914          8.2       0.2      295.0   10.0   

     u_wind    v_wind station  station_number                 time  latitude  \
0 -1.147153  1.638304    CWVK           73033  2018-05-01 12:00:00     50.28   
1 -1.209609  4.851479    CWVK           73033  2018-05-01 12:00:00     50.28   
2  8.485281  8.485281    CWVK           73033  2018-05-01 12:00:00     50.28   
3  7.727407  2.070552    CWVK           73033  2018-05-01 12:00:00     50.28   
4  9.063078 -4.226183    CWVK           73033  2018-05-01 12:00:00     50.28   

   longitude  elevation     pw  ddmmyyyy  
0    -119.27      3

In [16]:
df2 = df[["pw", "ddmmyyyy"]]
df3 = df[["speed", "ddmmyyyy"]]
df4 = df[["pressure", "speed", "ddmmyyyy"]]
df_tke = df[["height", "u_wind", "v_wind", "ddmmyyyy"]]
print(df_tke.head())

   height    u_wind    v_wind  ddmmyyyy
0     379 -1.147153  1.638304   1052018
1     440 -1.209609  4.851479   1052018
2     610  8.485281  8.485281   1052018
3     743  7.727407  2.070552   1052018
4     914  9.063078 -4.226183   1052018


In [7]:
# check the measurements per day
t = time.process_time()

print(df2
 .value_counts()
 .groupby("ddmmyyyy")
 .head(1)
 .reset_index(name="Count")
 )

elapsed_time = time.process_time() - t
print(elapsed_time)

        pw  ddmmyyyy  Count
0    14.79  15052018    157
1    13.67  10052018    150
2    13.29   6062018    146
3    17.43  25052018    146
4    21.98   5072018    146
..     ...       ...    ...
687   9.58  14072018     35
688   0.00   4062020     28
689   0.00  29062022     27
690   0.00   5052020     25
691  13.43  25062018     16

[692 rows x 3 columns]
0.039423032999999386


In [12]:
# wind the strongest mean wind each day
t = time.process_time()

print(df3
 .groupby(["ddmmyyyy"])
 .agg({"speed" : "mean"})
 .rename(columns={"speed": "mean"})
 .sort_values(["mean"], ascending=False)
 ) 

elapsed_time = time.process_time() - t
print(elapsed_time)

               mean
ddmmyyyy           
26092019  62.510204
1072018   53.938272
24092019  53.378947
25092019  50.802083
2062018   48.192771
...             ...
10052022   9.309278
5082018    8.927711
22082018   8.130000
21062018   7.041667
11092019   5.988372

[692 rows x 1 columns]
0.02621321099999996


In [13]:
# wind the strongest mean wind  each day
t = time.process_time()

print(df3
 .groupby(["ddmmyyyy"])
 .agg({"speed" : "mean"})
 .rename(columns={"speed": "mean"})
 .sort_values(["mean"], ascending=False)
 ) 

elapsed_time = time.process_time() - t
print(elapsed_time)

               mean
ddmmyyyy           
26092019  62.510204
1072018   53.938272
24092019  53.378947
25092019  50.802083
2062018   48.192771
...             ...
10052022   9.309278
5082018    8.927711
22082018   8.130000
21062018   7.041667
11092019   5.988372

[692 rows x 1 columns]
0.018288110999999496


In [19]:
# get lowest ~50mb of wind speed and get daily maximums
t = time.process_time()

df4 = df4.loc[df4["pressure"] > 940]
print(df4
      .groupby(["ddmmyyyy"])
      .agg({"speed" : "max"})
      .rename(columns={"speed": "max"})
      .sort_values(["max"], ascending=False)
     )

elapsed_time = time.process_time() - t
print(elapsed_time)

           max
ddmmyyyy      
3092018   21.0
20072018  21.0
8062022   18.0
28052021  17.0
19052018  16.0
...        ...
29082018   0.0
29082020   0.0
30062020   0.0
31052018   0.0
31052021   0.0

[688 rows x 1 columns]
0.024992697000000064


In [27]:
# find the mean tke in the lowest 1500m for each day
t = time.process_time()

df_tke = df_tke.loc[df_tke["height"] < 1500]  # all values wih height less than 
df_tke["u_var"] = np.nan  # primed values 
df_tke["v_var"] = np.nan

dates = df_tke["ddmmyyyy"].unique()

# get the daily variance values
uu = df_tke.groupby(["ddmmyyyy"]).agg({"u_wind" : "var"}).rename(columns={"u_wind" : "u_var"})
uu.set_index(dates)
vv = df_tke.groupby(["ddmmyyyy"]).agg({"v_wind" : "var"}).rename(columns={"v_wind" : "v_var"})
vv.set_index(dates)

# concatinate the two frames to calculate daily tke
frames = [uu, vv]
tke = pd.concat(frames, axis=1)
tke["tke"] = 0.5 * (tke["u_var"]**2 + tke["v_var"]**2)  # get half the square of both variance values

print(tke.sort_values(by=["tke"], ascending=False))

elapsed_time = time.process_time() - t
print(elapsed_time)   

ddmmyyyy
1052018     22.383788
1052019      4.805934
1052020      7.681580
1052021     34.038015
1052022      3.524046
              ...    
31072022    14.318889
31082018    21.199516
31082019     2.520502
31082020     9.681880
31082021     9.635348
Name: u_var, Length: 689, dtype: float64
               u_var       v_var          tke
ddmmyyyy                                     
25062018  117.634197   14.944404  7030.569771
15092021   18.866299  110.032897  6231.587796
16062018    3.814501   97.658267  4775.843795
4052022    83.874453   16.435040  3652.517193
7072018    85.196530    5.104336  3642.251526
...              ...         ...          ...
5082019     0.418119    0.889550     0.483061
2072022     0.386071    0.825657     0.415380
9072018     0.370044    0.520352     0.203849
31052022    0.421105    0.388474     0.164120
1072022     0.227515    0.297967     0.070274

[689 rows x 3 columns]
0.05590820200000124


In [28]:
# re-run the same commands with the cudf extension instead (dont have to change any syntax)
get_ipython().kernel.do_shutdown(restart=True)

{'status': 'ok', 'restart': True}

In [2]:
import cudf
%load_ext cudf.pandas

import pandas as pd
import os
import time
import numpy as np

The cudf.pandas extension is already loaded. To reload it, use:
  %reload_ext cudf.pandas


In [4]:
fpath = os.getcwd()  # assuming df is in current dir
df = pd.read_csv("12Z_soundings.csv")
print(df.head())

   Unnamed: 0  pressure  height  temperature  dewpoint  direction  speed  \
0           0     966.0     379         10.0       3.0      145.0    2.0   
1           1     959.0     440         11.0       3.0      166.0    5.0   
2           2     939.8     610         10.2       1.6      225.0   12.0   
3           3     925.0     743          9.6       0.6      255.0    8.0   
4           4     905.9     914          8.2       0.2      295.0   10.0   

     u_wind    v_wind station  station_number                 time  latitude  \
0 -1.147153  1.638304    CWVK           73033  2018-05-01 12:00:00     50.28   
1 -1.209609  4.851479    CWVK           73033  2018-05-01 12:00:00     50.28   
2  8.485281  8.485281    CWVK           73033  2018-05-01 12:00:00     50.28   
3  7.727407  2.070552    CWVK           73033  2018-05-01 12:00:00     50.28   
4  9.063078 -4.226183    CWVK           73033  2018-05-01 12:00:00     50.28   

   longitude  elevation     pw  ddmmyyyy  
0    -119.27      3

In [5]:
df2 = df[["pw", "ddmmyyyy"]]
df3 = df[["speed", "ddmmyyyy"]]
df4 = df[["pressure", "speed", "ddmmyyyy"]]
df_tke = df[["height", "u_wind", "v_wind", "ddmmyyyy"]]
print(df_tke.head())

   height    u_wind    v_wind  ddmmyyyy
0     379 -1.147153  1.638304   1052018
1     440 -1.209609  4.851479   1052018
2     610  8.485281  8.485281   1052018
3     743  7.727407  2.070552   1052018
4     914  9.063078 -4.226183   1052018


In [6]:
# wind the strongest mean wind  each day
t = time.process_time()

print(df3
 .groupby(["ddmmyyyy"])
 .agg({"speed" : "mean"})
 .rename(columns={"speed": "mean"})
 .sort_values(["mean"], ascending=False)
 ) 

elapsed_time = time.process_time() - t
print(elapsed_time)

               mean
ddmmyyyy           
26092019  62.510204
1072018   53.938272
24092019  53.378947
25092019  50.802083
2062018   48.192771
...             ...
10052022   9.309278
5082018    8.927711
22082018   8.130000
21062018   7.041667
11092019   5.988372

[692 rows x 1 columns]
0.07968894999999954


In [8]:
# find the mean tke in the lowest 1500m for each day
t = time.process_time()

df_tke = df_tke.loc[df_tke["height"] < 1500]  # all values wih height less than 
df_tke["u_var"] = np.nan  # primed values 
df_tke["v_var"] = np.nan

dates = df_tke["ddmmyyyy"].unique()

# get the daily variance values
uu = df_tke.groupby(["ddmmyyyy"]).agg({"u_wind" : "var"}).rename(columns={"u_wind" : "u_var"})
uu.set_index(dates)
vv = df_tke.groupby(["ddmmyyyy"]).agg({"v_wind" : "var"}).rename(columns={"v_wind" : "v_var"})
vv.set_index(dates)

# concatinate the two frames to calculate daily tke
frames = [uu, vv]
tke = pd.concat(frames, axis=1)
tke["tke"] = 0.5 * (tke["u_var"]**2 + tke["v_var"]**2)  # get half the square of both variance values

print(tke.sort_values(by=["tke"], ascending=False))

elapsed_time = time.process_time() - t
print(elapsed_time) 

               u_var       v_var          tke
ddmmyyyy                                     
25062018  117.634197   14.944404  7030.569771
15092021   18.866299  110.032897  6231.587796
16062018    3.814501   97.658267  4775.843795
4052022    83.874453   16.435040  3652.517193
7072018    85.196530    5.104336  3642.251526
...              ...         ...          ...
5082019     0.418119    0.889550     0.483061
2072022     0.386071    0.825657     0.415380
9072018     0.370044    0.520352     0.203849
31052022    0.421105    0.388474     0.164120
1072022     0.227515    0.297967     0.070274

[689 rows x 3 columns]
0.040793405000000504
